In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression

# Para separar el el dataframe en test de validación y test de entrenamiento
from sklearn.model_selection import train_test_split
# Para hacer varias evaluaciones del modelo con un conjunto de validación 
from sklearn.model_selection import cross_val_score
# Métricas
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
# Modelos
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC

from nltk import word_tokenize

In [3]:
# Cargar los datos
df_train = pd.read_excel('C:/Users/gerrodri/Documents/Academia/insumos/correos.xlsx')
df_val = pd.read_excel('C:/Users/gerrodri/Documents/Academia/insumos/nuevos_correos.xlsx')

df_val["Subject"] = df_val["Subject"].astype(str)
df_val["Preview"] = df_val["Preview"].astype(str)

In [4]:
# Convertir la columna 'DateTimeSent' a tipo datetime
df_val['DateTimeSent'] = pd.to_datetime(df_val['DateTimeSent'])

# Obtener el mes actual
mes_actual = pd.to_datetime('today').month
print(mes_actual)

10


In [5]:
#  Eliminar las filas que no corresponden al mes actual
df_val = df_val[~(df_val['DateTimeSent'].dt.month != mes_actual)]

In [6]:
# Crear una tupla con las palabras o frases a eliminar
palabras_a_eliminar = ("Solicitud atendida", "Notificación Finalizaciòn", "RPA", 
                       "El buzón está lleno.")

# Crear una función para determinar si una fila debe eliminarse
def debe_eliminar(fila):
    for palabra in palabras_a_eliminar:
        if palabra in fila["Subject"]:
            return True
    return False

# Eliminar las filas que cumplan con el criterio
df_val = df_val[~df_val.apply(debe_eliminar, axis=1)]

In [7]:
# Define una función para transformar la columna "Clasificación"
def transform_clasificacion(clasificacion):
    if clasificacion == "Certificación Bancaria Basica":
        return clasificacion
    else:
        return "otros"
    
# Aplica la función a la columna "Clasificación" y crea una nueva columna "Clasificación Nueva"
df_train['Clasificación'] = df_train['Clasificación'].apply(transform_clasificacion)

In [8]:
# Normalizar el texto
def normalize_text(text):
    text = text.lower()
    text = text.replace("á", "a")
    text = text.replace("é", "e")
    text = text.replace("í", "i")
    text = text.replace("ó", "o")
    text = text.replace("ú", "u")

    return text

In [9]:
# Aplicar la normalización al dataset de entrenamiento
df_train["Subject"] = df_train["Subject"].apply(normalize_text)
df_train["Preview"] = df_train["Preview"].apply(normalize_text)

# Aplicar la normalización al dataset de validación
df_val["Subject"] = df_val["Subject"].apply(normalize_text)
df_val["Preview"] = df_val["Preview"].apply(normalize_text)

In [10]:
df_train["text"] = df_train["Subject"] + df_train["Preview"]

df_val["text"] = df_val["Subject"] + df_val["Preview"]

In [11]:
df_train["Clasificación"] = df_train["Clasificación"].apply(lambda x: 0 if x == "otros" else 1)

In [12]:
df_train["Clasificación"].drop_duplicates()

0    0
1    1
Name: Clasificación, dtype: int64

In [13]:
cv = CountVectorizer()
X_train = cv.fit_transform(df_train["text"])

X_val = cv.transform(df_val["text"])

In [14]:
# Aplicar un TfidfTransformer
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
# X_test = tfidf.fit_transform(X_test)

X_val = tfidf.transform(X_val)

In [15]:
sgd_clf = SGDClassifier(random_state=42)
df_clasificado_sgd = cross_val_predict(sgd_clf, X_train, df_train["Clasificación"], cv=3)
confusion_matrix(df_train["Clasificación"], df_clasificado_sgd)

array([[ 431,   22],
       [  15, 1175]], dtype=int64)

In [16]:
precision_score(df_train["Clasificación"].to_numpy(), df_clasificado_sgd, pos_label=1)

0.9816207184628237

In [17]:
recall_score(df_train["Clasificación"], df_clasificado_sgd, pos_label=0)

0.9514348785871964

In [18]:
forest_reg = SGDClassifier(random_state=42)
df_clasificado_forest = cross_val_predict(forest_reg, X_train, df_train["Clasificación"], cv=3)
confusion_matrix(df_train["Clasificación"], df_clasificado_forest)


array([[ 431,   22],
       [  15, 1175]], dtype=int64)

In [19]:
precision_score(df_train["Clasificación"].to_numpy(), df_clasificado_forest, pos_label=1)

0.9816207184628237

In [20]:
recall_score(df_train["Clasificación"], df_clasificado_forest, pos_label=0)

0.9514348785871964

In [21]:
tree_reg = DecisionTreeRegressor()
df_clasificado_tree = cross_val_predict(tree_reg, X_train, df_train["Clasificación"], cv=3)
confusion_matrix(df_train["Clasificación"], df_clasificado_tree)

array([[ 430,   23],
       [  15, 1175]], dtype=int64)

In [22]:
precision_score(df_train["Clasificación"].to_numpy(), df_clasificado_tree, pos_label=1)

0.9808013355592654

In [23]:
recall_score(df_train["Clasificación"], df_clasificado_tree, pos_label=0)

0.9492273730684326

In [24]:
svm_classifier = SVC(kernel='linear')
df_clasificado_svm = cross_val_predict(svm_classifier, X_train, df_train["Clasificación"], cv=3)
confusion_matrix(df_train["Clasificación"], df_clasificado_svm)

array([[ 436,   17],
       [  15, 1175]], dtype=int64)

In [25]:
precision_score(df_train["Clasificación"].to_numpy(), df_clasificado_svm, pos_label=1)

0.985738255033557

In [26]:
# Instanciar el modelo de regresión logística
model = LogisticRegression()
df_clasificado_log_reg = cross_val_predict(model, X_train, df_train["Clasificación"], cv=3)
# Entrenar el modelo
confusion_matrix(df_train["Clasificación"], df_clasificado_log_reg)

array([[ 389,   64],
       [  15, 1175]], dtype=int64)

In [27]:
precision_score(df_train["Clasificación"].to_numpy(), df_clasificado_log_reg, pos_label=1)

0.9483454398708636

In [28]:
# Predecir las etiquetas del archivo de validación
sgd_clf.fit(X_train, df_train["Clasificación"])
y_pred = sgd_clf.predict(X_val)
# Agregar la columna clasificación al archivo de validación
df_val["clasificacion"] = y_pred

df_val.to_excel('C:/Users/gerrodri/Documents/Academia/out/val_sgd.xlsx')

In [30]:
forest_reg.fit(X_train, df_train["Clasificación"])
y_pred = forest_reg.predict(X_val)
# Agregar la columna clasificación al archivo de validación
df_val["clasificacion"] = y_pred

df_val.to_excel('C:/Users/gerrodri/Documents/Academia/out/val_random_forest.xlsx')

In [31]:
tree_reg.fit(X_train, df_train["Clasificación"])
y_pred = tree_reg.predict(X_val)
# Agregar la columna clasificación al archivo de validación
df_val["clasificacion"] = y_pred

df_val.to_excel('C:/Users/gerrodri/Documents/Academia/out/val_arbol_decisión.xlsx')

In [32]:
svm_classifier.fit(X_train, df_train["Clasificación"])
y_pred = svm_classifier.predict(X_val)
# Agregar la columna clasificación al archivo de validación
df_val["clasificacion"] = y_pred

df_val.to_excel('C:/Users/gerrodri/Documents/Academia/out/val_svm.xlsx')


In [33]:
model.fit(X_train, df_train["Clasificación"])
y_pred = model.predict(X_val)
# Agregar la columna clasificación al archivo de validación
df_val["clasificacion"] = y_pred

df_val.to_excel('C:/Users/gerrodri/Documents/Academia/out/val_log_reg.xlsx')